# Purpose of this notebook
An obstacle for getting started with Neuroglancer is the data formats you can feed it are somewhat unfamiliar. For example, it does not accept TIFF format. It does this for a good reason though which is to be more efficient. One of the formats it accepts is called "precomputed" format, and that is the one we will use in this notebook. Fortunately there is a python pipeline for making precomputed data from TIFF files. 

Neuroglancer also is not set up to read CSV files. Instead, it uses the JSON file format, which essentially look like python dictionaries.

This notebook covers how to convert a custom annotated atlas volume (of the same format as the 'WHS_SD_rat_atlas_v3_annotation.tif' file on bucket) to precomputed format so that you can load it in to Neuroglancer. It then covers how to make a 3d mesh from this volume so that the atlas can be viewed in the 3d viewer within Neuroglancer. It also covers how to convert a CSV file formatted like the 'labels_v3.csv' file on bucket containing the region name mapping for the Rat MRI atlas into JSON format so that Neuroglancer can read it and display the region names.

In the following I use the original MRI atlas annotation volume and CSV file as an example. To use custom annotation volume and/or custom CSV file, replace the variables at the top of the notebook with your custom ones. The rest of the notebook should not (hopefully!) need to be changed before running.

## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like what you would get as output from the light sheet microscope) or type "segmentation" (like an atlas annotation volume). The naming is a little confusing because both layer types refer to volumes (3-d objects). In this notebook, we are only concerned with a single layer: the annotation volume, which is a segmentation layer. If you were to make multiple annotation volumes (with different boundaries, etc.), each one of those would be a different layer. In Neuroglancer, you can overlay multiple layers or view them side-by-side. 

# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment "ng_mriatlas" can be set up in the following way:
In terminal:
- conda create -n ng_mriatlas python=3.7.4 -y
- conda activate ng_mriatlas # (or source activate ng_mriatlas, depending on which version of conda you have)
- pip install cloud-volume
- pip install SimpleITK
- **pip install neuroglancer==2.8** <br>

\# Optional: if you want 3d meshing following the next steps
- git clone https://github.com/seung-lab/igneous.git igneous
- cd igneous
- pip install -r requirements.txt 
- python setup.py develop

\# To enable you to use jupyter notebooks to work with this environment as a kernel:
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng_mriatlas

Once this is all installed, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the notebook server)

In [1]:
import os,csv,json
import numpy as np
from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
import SimpleITK as sitk

from concurrent.futures import ProcessPoolExecutor

import neuroglancer
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc


In [7]:

# Point to the custom annotation volume file that you have modified (in this example we point to the original one) 
custom_vol_path = '/home/emilyjanedennis/Desktop/k310_1_1x_555_016na_1hfds_z10um_50msec_20povlp_resized_ch00.tif'
layer_dir = '/home/emilyjanedennis/Desktop/GitHub/ng/k310_555'
mkdir(layer_dir)
print(f"created {layer_dir}")
cpus_to_use = 8

created /home/emilyjanedennis/Desktop/GitHub/ng/k310_555


## Step 2: Write the instructions ("info") file that will tell Neuroglancer about your annotation volume and the annotation label file you just created
In the last step we created an info file inside the "segment_properties" subdirectory of our layer. Now we need to create an info file in the layer directory itself. This info file is also a JSON file, but it contains more general things about our layer like the shape and physical resolution of the volume. It will also have an instruction on where to find our segment properties info file that we just made. 

In [8]:
def make_info_file(resolution_xyz,volume_size_xyz,layer_dir):
    """ Make an JSON-formatted file called the "info" file
    for use with the precomputed data format. 
    Precomputed is one of the formats that Neuroglancer can read in.  
    --- parameters ---
    resolution_xyz:      A tuple representing the size of the pixels (dx,dy,dz) 
                         in nanometers, e.g. (20000,20000,5000) for 20 micron x 20 micron x 5 micron
    
    volume_size_xyz:     A tuple representing the number of pixels in each dimension (Nx,Ny,Nz)
    
                         
    layer_dir:           The directory where the precomputed data will be
                         saved
    """
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'image', # 'image' or 'segmentation'
        data_type = 'uint16', # 32 not necessary for atlases unless you have > 2^(32)-1 labels  
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = resolution_xyz, # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels.
        volume_size = volume_size_xyz, # X,Y,Z size in voxels
    )

    vol = CloudVolume(f'file://{layer_dir}', info=info)
    vol.provenance.description = "A test info file" # can change this if you want a description
    vol.provenance.owners = [''] # list of contact email addresses
    # Saves the info and provenance files for the first time
    vol.commit_info() # generates info json file
    vol.commit_provenance() # generates provenance json file
    print("Created CloudVolume info file: ",vol.info_cloudpath)

    return vol

In [9]:
## Make the info file

#20 micron resolution in nm
resolution_xyz = (10000,10000,10000)
# Load the volume into memory and get its shape 
annotation_vol = np.array(sitk.GetArrayFromImage(
    sitk.ReadImage(custom_vol_path)),dtype=np.uint16,order='F')
z_dim,y_dim,x_dim = annotation_vol.shape
volume_size_xyz = (x_dim,y_dim,z_dim)

# Write the info file
vol = make_info_file(
    resolution_xyz=resolution_xyz,
    volume_size_xyz=volume_size_xyz,
    layer_dir=layer_dir)


Created CloudVolume info file:  file:///home/emilyjanedennis/Desktop/GitHub/ng/k310_555/info


# Step 3: Convert annotation volume to precomputed data format
First we create a directory (the "progress_dir") at the same folder level as the layer directory to keep track of the progress of the conversion. 
All the conversion does is copy the numpy array representing the 3d volume to a new object "vol". This is done one plane at a time (although it is parallelized). As each plane is converted, an empty file is created in the progress_dir with the name of the plane. By the end of the conversion, there should be as many files in this progress_dir as there are z planes. 

In [10]:
layer_name = layer_dir.split('/')[-1]
parent_dir = '/'.join(layer_dir.split('/')[:-1])
progress_dir = mkdir(parent_dir+ f'/progress_{layer_name}') # unlike os.mkdir doesn't crash on prexisting 
print(f"created directory: {progress_dir}")

created directory: /home/emilyjanedennis/Desktop/GitHub/ng/progress_k310_555


In [11]:
def process_slice(z):
    """ This function copies a 2d image slice from the atlas volume
    to the cloudvolume object, vol. We will run this in parallel over 
    all z planes
    ---parameters---
    z:    An integer representing the 0-indexed z plane to be converted
    """
    if os.path.exists(os.path.join(progress_dir, str(z))):
        print(f"Slice {z} already processed, skipping ")
        return
    if z >= z_dim: # z is zero indexed and runs from 0-(z_dim-1)
        print("Index {z} >= z_dim of volume, skipping")
        return
    print('Processing slice z=',z)
    array = annotation_vol[z].reshape((1,y_dim,x_dim)).T
    vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))
    return "success"


In [12]:
# Run the conversion in parallel. It's not a huge amount of processing but the more cores the better

# First figure out if there are any planes that have already been converted 
# by checking the progress dir
done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(vol.bounds.minpt.z, vol.bounds.maxpt.z))
# Figure out the ones we still need to convert 
to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
print(f"Have {len(to_upload)} planes to upload")
with ProcessPoolExecutor(max_workers=cpus_to_use) as executor:
    for result in executor.map(process_slice,to_upload):
        try:
            print(result)
        except Exception as exc:
            print(f'generated an exception: {exc}')

Have 1048 planes to upload
Processing slice z=Processing slice z=Processing slice z=Processing slice z=Processing slice z=Processing slice z=Processing slice z= Processing slice z=     3 14 6
5
207







Uploading: 100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


Processing slice z= 8


Uploading:   0%|          | 0/2 [00:00<?, ?it/s]

success


...

success
success


Uploading: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]

Uploading: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


success
success
success


Uploading: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]


success
success
success


# Step 5: Host the precomputed data on your machine so that Neuroglancer can see it
This step is really easy! Note: Exectuing the code below will cause your jupyter notebook to hang, so it is better to run the following code in a new ipython terminal (make sure to have the ng_mriatlas conda environment activated in that python session) rather than the notebook. 

```python
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)
```

# Step 6: View your custom volume and labels in Neuroglancer
Step 4 hosts your data via http on port 1338 of your local machine. To actually view your data in Neuroglancer, there are two ways to do this. You can either load the data in manually in the browser or load it in with python. 

For the manual method, open up the Braincogs Neuroglancer client: [https://nglancer.pni.princeton.edu](https://nglancer.pni.princeton.edu) (you must be using a Princeton VPN) and then click the "+" in the upper left hand corner of the screen once the black screen loads. To load in your data, type the following into the source text box:<br>
> precomputed://http://localhost:1338 <br>

Then hit tab and name your layer if you'd like. Hit enter or the "add layer" button and your layer should load into Neuroglancer. Hopefully the labels you added should be showing up in the bottom left when you hover over a region. 

For the python method, you can do this by executing the following cells. Make sure you have hosted the data in another python instance somewhere on your local machine at port 1338.